In [1]:
from mbi import CliqueVector, GraphicalModel, Dataset, Domain, Factor
import numpy as np
from scipy import optimize

In [9]:
def learn_model_exact(domain, total, marginals):
    data_marginals = CliqueVector(marginals)*(1.0/total)
    original_cliques = list(marginals.keys())
    model = GraphicalModel(domain, original_cliques, 1.0)
    if len(marginals) == 0:
        model.potentials = CliqueVector.zeros(domain, model.cliques)
        model.marginals = model.belief_propagation(model.potentials)
        model.total = total
        return model
    index = {}
    idx = 0
    for cl in original_cliques:
        end = idx + domain.size(cl)
        index[cl] = (idx, end)
        idx = end

    def to_vector(mu):
        return np.concatenate([mu[cl].datavector() for cl in original_cliques])

    def to_cliquevector(vector):
        marginals = {}
        for cl in original_cliques:
            start, end = index[cl]
            dom = domain.project(cl)
            marginals[cl] = Factor(dom, vector[start:end])
        return CliqueVector(marginals)
    
    def expand(pot):
        potentials = CliqueVector.zeros(domain, model.cliques)
        for cl in pot:
            for cl2 in model.cliques:
                if set(cl) <= set(cl2):
                    potentials[cl2] += pot[cl]
                    break
        return potentials
    
    def contract(mu):
        mu2 = {}
        for cl in original_cliques:
            for cl2 in mu:
                if set(cl) <= set(cl2):
                    mu2[cl] = mu[cl2].project(cl)
                    break
        return CliqueVector(mu2)

    def loss_and_grad(potentials):
        pot = to_cliquevector(potentials)
        pot2 = expand(pot)
        logZ = model.belief_propagation(pot2, logZ=True)
        model_marginals = contract(model.belief_propagation(pot2))
        
        #print(pot, model_marginals)

        loss = pot.dot(data_marginals) - logZ
        grad = data_marginals + -1*model_marginals
        return -loss, -to_vector(grad)

    x0 = to_vector(CliqueVector.zeros(domain, original_cliques))
    ans = optimize.minimize(loss_and_grad, x0=x0, method='L-BFGS-B', jac=True)
    print(ans)
    
    model.total = total
    model.potentials = expand(to_cliquevector(ans['x']))
    model.marginals = model.belief_propagation(model.potentials)
    return model


In [12]:
dom = Domain(['A','B','C'],[2,3,4])
n = 100
data = Dataset.synthetic(dom,n)

marginals = {}
cliques = [('A','B')]
for cl in cliques:
    mu = data.project(cl)
    marginals[cl] = Factor(mu.domain, mu.datavector())
    
model = learn_model_exact(dom, data.records, marginals)
print(model.potentials)
for cl in cliques:
    print(model.project(cl).datavector())
    print(data.project(cl).datavector())
    print()

      fun: 3.1294905414016063
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.50271793e-06, -6.80845900e-07, -1.18532770e-06, -6.80845900e-07,
        2.19928739e-06,  1.85045004e-06])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 7
      nit: 6
     njev: 7
   status: 0
  success: True
        x: array([-0.28343234, -0.20338234,  0.00425485, -0.20338234,  0.12205751,
        0.56388466])
{('A', 'B'): <mbi.factor.Factor object at 0x7fb94ddd9b80>, ('C',): <mbi.factor.Factor object at 0x7fb94ddd9760>}
[11.99984973 12.99993192 15.99988147 12.99993192 18.00021993 28.00018505]
[12. 13. 16. 13. 18. 28.]

